In [8]:
symbols = ['VIC', 'VNM','HPG', 'VCB', 'BID', 'CTG', 'MSN', 'FPT', 'MWG', 'GAS', 'VHM', 'BVH']

In [9]:
def backtest_sma(symbol, stop_loss_pct=0.05):
    from vnstock import Vnstock
    import talib
    import pandas as pd
    import numpy as np

    try:
        stock = Vnstock().stock(symbol=symbol, source='VCI')
        df = stock.quote.history(start='2020-03-30', end='2025-03-30', interval='1D')
        df['time'] = pd.to_datetime(df['time'])
        df.set_index('time', inplace=True)
        
        df['SMA_10'] = talib.SMA(df['close'], timeperiod=10)
        df['SMA_20'] = talib.SMA(df['close'], timeperiod=20)
        
        df['Signal'] = 0
        df.loc[df.index[10:], 'Signal'] = (df['SMA_10'].iloc[20:] > df['SMA_20'].iloc[20:]).astype(int)
        df['Position'] = df['Signal'].diff()

        trades = []
        in_trade = False
        buy_price = None
        buy_date = None
        account = 0
        total_invested = 0
        for index, row in df.iterrows():
            if row['Position'] == 1.0 and not in_trade:
                buy_price = row['close']
                buy_date = index
                if account < buy_price:
                    need_to_add = buy_price - account
                    total_invested += need_to_add
                    account += need_to_add

                account -= buy_price
                in_trade = True
            elif in_trade:
                current_price = row['close']
                if current_price <= buy_price * (1 - stop_loss_pct):
                    account += current_price
                    trades.append({'Buy_Date': buy_date, 
                                   'Buy_Price': buy_price,
                                   'Sell_Date': index, 
                                   'Sell_Price': current_price,
                                   'Profit': current_price - buy_price, 
                                   'Reason': 'Stop Loss'})
                    in_trade = False
                elif row['Position'] == -1.0:
                    account += current_price
                    trades.append({'Buy_Date': buy_date, 
                                   'Buy_Price': buy_price,
                                   'Sell_Date': index, 
                                   'Sell_Price': current_price,
                                   'Profit': current_price - buy_price, 
                                   'Reason': 'Signal Sell'})
                    in_trade = False

        df_trades = pd.DataFrame(trades)
        df_trades['Return_%'] = df_trades['Profit'] / df_trades['Buy_Price'] * 100
        #total_return = df_trades['Profit'].sum() / df_trades['Buy_Price'].iloc[0] * 100
        net_profit = account - total_invested
        total_return = (net_profit / total_invested * 100)
        winrate = len(df_trades[df_trades['Profit'] > 0]) / len(df_trades) * 100 #if not df_trades.empty else 0

        return {
            'Symbol': symbol,
            'Trades': len(df_trades),
            'Total_Return(%)': round(total_return, 2),
            'Winrate(%)': round(winrate, 2),
            'Average_Return_Per_Trade': round(total_return / len(df_trades), 2) if not df_trades.empty else 0,
            'Net_Profit': net_profit,
            'Total_Invested': total_invested
        }

    except Exception as e:
        return {'Symbol': symbol, 'Error': str(e)}


In [10]:
results = []

for sym in symbols:
    result = backtest_sma(sym)
    results.append(result)

import pandas as pd
df_results = pd.DataFrame(results)
print(df_results)


   Symbol  Trades  Total_Return(%)  Winrate(%)  Average_Return_Per_Trade  \
0     VIC      32           -62.19       25.00                     -1.94   
1     VNM      36           -31.33       27.78                     -0.87   
2     HPG      31           175.17       48.39                      5.65   
3     VCB      37          -100.00       32.43                     -2.70   
4     BID      36            37.87       36.11                      1.05   
5     CTG      33            72.07       54.55                      2.18   
6     MSN      38           -22.68       31.58                     -0.60   
7     FPT      31            71.19       45.16                      2.30   
8     MWG      31            13.22       41.94                      0.43   
9     GAS      34            19.56       44.12                      0.58   
10    VHM      36           -90.56       25.00                     -2.52   
11    BVH      36            -2.29       44.44                     -0.06   

    Net_Pro

In [11]:
def backtest_macd(symbol, stop_loss_pct=0.05):
    from vnstock import Vnstock
    import talib
    import pandas as pd

    try:
        stock = Vnstock().stock(symbol=symbol, source='VCI')
        df = stock.quote.history(start='2020-03-30', end='2025-03-30', interval='1D')
        df['time'] = pd.to_datetime(df['time'])
        df.set_index('time', inplace=True)

        df['EMA_12'] = talib.EMA(df['close'], timeperiod=12)
        df['EMA_26'] = talib.EMA(df['close'], timeperiod=26)
        df['MACD'], df['MACD_signal'], _ = talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)

        df['Signal'] = 0
        df.loc[df.index[26:], 'Signal'] = (df['MACD'].iloc[26:] > df['MACD_signal'].iloc[26:]).astype(int)
        df['Position'] = df['Signal'].diff()

        trades = []
        in_trade = False
        buy_price = None
        buy_date = None
        account = 0
        total_invested = 0
        for index, row in df.iterrows():
            if row['Position'] == 1.0 and not in_trade:
                buy_price = row['close']
                buy_date = index
                if account < buy_price:
                    need_to_add = buy_price - account
                    total_invested += need_to_add
                    account += need_to_add

                account -= buy_price
                in_trade = True
            elif in_trade:
                current_price = row['close']
                if current_price <= buy_price * (1 - stop_loss_pct):
                    account += current_price
                    trades.append({
                        'Buy_Date': buy_date, 'Buy_Price': buy_price,
                        'Sell_Date': index, 'Sell_Price': current_price,
                        'Profit': current_price - buy_price,
                        'Reason': 'Stop Loss'
                    })
                    in_trade = False
                elif row['Position'] == -1.0:
                    account += current_price
                    trades.append({
                        'Buy_Date': buy_date, 'Buy_Price': buy_price,
                        'Sell_Date': index, 'Sell_Price': current_price,
                        'Profit': current_price - buy_price,
                        'Reason': 'Signal Sell'
                    })
                    in_trade = False

        df_trades = pd.DataFrame(trades)
        df_trades['Return_%'] = df_trades['Profit'] / df_trades['Buy_Price'] * 100
        #total_return = df_trades['Profit'].sum() / df_trades['Buy_Price'].iloc[0] * 100
        net_profit = account - total_invested
        total_return = (net_profit / total_invested * 100)
        winrate = len(df_trades[df_trades['Profit'] > 0]) / len(df_trades) * 100

        return {
            'Symbol': symbol,
            'Trades': len(df_trades),
            'Total_Return(%)': round(total_return, 2),
            'Winrate(%)': round(winrate, 2),
            'Average_Return_Per_Trade': round(total_return / len(df_trades), 2) if not df_trades.empty else 0,
            'Net_Profit': net_profit,
            'Total_Invested': total_invested
        }

    except Exception as e:
        return {'Symbol': symbol, 'Error': str(e)}


In [12]:
results_macd = []

for sym in symbols:
    result_macd = backtest_macd(sym)
    results_macd.append(result_macd)

import pandas as pd
df_results_macd = pd.DataFrame(results_macd)
print(df_results_macd)


   Symbol  Trades  Total_Return(%)  Winrate(%)  Average_Return_Per_Trade  \
0     VIC      45           -35.55       35.56                     -0.79   
1     VNM      51           -27.47       27.45                     -0.54   
2     HPG      43            67.78       48.84                      1.58   
3     VCB      54            12.65       37.04                      0.23   
4     BID      44            13.97       34.09                      0.32   
5     CTG      47            31.90       40.43                      0.68   
6     MSN      45            16.94       37.78                      0.38   
7     FPT      54            26.06       40.74                      0.48   
8     MWG      43            24.67       46.51                      0.57   
9     GAS      43            68.17       39.53                      1.59   
10    VHM      45           -50.95       28.89                     -1.13   
11    BVH      54            -2.68       38.89                     -0.05   

    Net_Pro

In [13]:
def backtest_macd_divergence(symbol, width=30):
    from vnstock import Vnstock
    import talib
    import pandas as pd
    import numpy as np

    try:
        stock = Vnstock().stock(symbol=symbol, source='VCI')
        df = stock.quote.history(start='2020-03-30', end='2025-03-30', interval='1D')
        df['time'] = pd.to_datetime(df['time'])
        df.set_index('time', inplace=True)

        df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
        df['Bullish_divergence'] = 0
        df['Bearish_divergence'] = 0
        df['Buy_price'] = np.nan
        df['Sell_price'] = np.nan

        for i in range(len(df)):
            try:
                if df.iloc[i]['MACD_hist'] < 0 and df.iloc[i]['MACD_hist'] == df.iloc[i:i+width]['MACD_hist'].min():
                    for r in range(i+10, min(i+width, len(df))):
                        if df.iloc[r]['MACD_hist'] < 0 and df.iloc[r]['low'] < df.iloc[i]['low'] and df.iloc[r]['MACD_hist'] - df.iloc[i]['MACD_hist'] > 0.02:
                            for s in range(r+1, min(r+width, len(df))):
                                if df.iloc[s]['MACD_hist'] > df.iloc[r]['MACD_hist'] and df.iloc[s]['MACD_hist'] < 0:
                                    df.iloc[s+1, df.columns.get_loc('Bullish_divergence')] = 1
                                    df.iloc[s+1, df.columns.get_loc('Buy_price')] = df.iloc[s+1]['close']
                                    #i=s+1
                                    break
            except IndexError:
                continue
                #pass

        for i in range(len(df)):
            try:
                if df.iloc[i]['MACD_hist'] > 0 and df.iloc[i]['MACD_hist'] == df.iloc[i:i+width]['MACD_hist'].max():
                    for r in range(i+10, min(i+width, len(df))):
                        if df.iloc[r]['MACD_hist'] > 0 and df.iloc[r]['high'] > df.iloc[i]['high'] and df.iloc[i]['MACD_hist'] - df.iloc[r]['MACD_hist'] > 0.02:
                            for s in range(r+1, min(r+width, len(df))):
                                if df.iloc[s]['MACD_hist'] < df.iloc[r]['MACD_hist'] and df.iloc[s]['MACD_hist'] > 0:
                                    df.iloc[s+1, df.columns.get_loc('Bearish_divergence')] = -1
                                    df.iloc[s+1, df.columns.get_loc('Sell_price')] = df.iloc[s+1]['close']
                                    #i=s+1
                                    break
            except IndexError:
                continue
                #pass

        df['Position'] = 0
        df.loc[df['Bullish_divergence'] == 1, 'Position'] = 1
        df.loc[df['Bearish_divergence'] == -1, 'Position'] = -1

        df_signals = df[df['Position'].isin([1, -1])][['close', 'MACD_hist', 'Position']]
        account_div = 0
        total_invested_div = 0
        pending_buys = []
        trades = []

        for index, row in df_signals.iterrows():
            if row['Position'] == 1:
                pending_buys.append((index, row['close']))
            elif row['Position'] == -1 and pending_buys:
                buy_date, buy_price = pending_buys.pop(0)
                sell_price = row['close']
                sell_date = index
                profit = sell_price - buy_price

                if account_div < buy_price:
                    need_to_add = buy_price - account_div
                    total_invested_div += need_to_add
                    account_div += need_to_add

                account_div -= buy_price
                account_div += sell_price

                trades.append({
                    'Buy_Date': buy_date,
                    'Buy_Price': buy_price,
                    'Sell_Date': sell_date,
                    'Sell_Price': sell_price,
                    'Profit': profit
                })

        df_trades = pd.DataFrame(trades)
        df_trades['Return_%'] = df_trades['Profit'] / df_trades['Buy_Price'] * 100

        net_profit = account_div - total_invested_div
        total_return = (net_profit / total_invested_div) * 100 if total_invested_div > 0 else 0
        win_rate = len(df_trades[df_trades['Profit'] > 0]) / len(df_trades) * 100 if len(df_trades) > 0 else 0
        avg_return = total_return / len(df_trades) if len(df_trades) > 0 else 0

        return {
            'Symbol': symbol,
            'Trades': len(df_trades),
            'Total Return(%)': round(total_return, 2),
            'Winrate(%)': round(win_rate, 2),
            'Average Return Per Trade': round(avg_return, 2),
            'Net_Profit': net_profit,
            'Total_Invested': total_invested_div
        }

    except Exception as e:
        return {'Symbol': symbol, 'Error': str(e)}


In [14]:
results_div = []

for sym in symbols:
    result_div = backtest_macd_divergence(sym)
    results_div.append(result_div)

import pandas as pd
df_results_div = pd.DataFrame(results_div)
print(df_results_div)


   Symbol  Trades  Total Return(%)  Winrate(%)  Average Return Per Trade  \
0     VIC      43           -70.52       39.53                     -1.64   
1     VNM      42           -59.60       38.10                     -1.42   
2     HPG      40            18.25       47.50                      0.46   
3     VCB      57           941.07       85.96                     16.51   
4     BID      65          1348.40       84.62                     20.74   
5     CTG      36           143.49       72.22                      3.99   
6     MSN      41           116.87       39.02                      2.85   
7     FPT      39           299.95       61.54                      7.69   
8     MWG      65           674.78       76.92                     10.38   
9     GAS      51           453.77       68.63                      8.90   
10    VHM      35           -71.42       40.00                     -2.04   
11    BVH      44           171.59       70.45                      3.90   

    Net_Pro

In [15]:
def compute_total_return(df, strategy_name):
    df_valid = df[~df['Net_Profit'].isna() & ~df['Total_Invested'].isna()]
    total_profit = df_valid['Net_Profit'].sum()
    total_invested = df_valid['Total_Invested'].sum()
    total_return_pct = (total_profit / total_invested * 100) if total_invested > 0 else 0
    return {
        'Strategy': strategy_name,
        'Total_Profit': round(total_profit, 2),
        'Total_Invested': round(total_invested, 2),
        'Total_Return(%)': round(total_return_pct, 2)
    }


In [16]:
summary_sma = compute_total_return(df_results, 'SMA')
summary_macd = compute_total_return(df_results_macd, 'MACD')
summary_macd_div = compute_total_return(df_results_div, 'MACD Divergence')

import pandas as pd
df_summary = pd.DataFrame([summary_sma, summary_macd, summary_macd_div])
print(df_summary)


          Strategy  Total_Profit  Total_Invested  Total_Return(%)
0              SMA       -130.03          810.67           -16.04
1             MACD         44.53          825.61             5.39
2  MACD Divergence        935.79         1158.22            80.80
